Import necessary modules

In [ ]:
import numpy as np
import scipy as sp
import scipy.io as sio
import scipy.signal as sig
import pywt
import os
import glob
import itertools
import matplotlib
import pandas as pd
import re
import math
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
import tensorflow.contrib.rnn as recurrent
import sklearn.preprocessing
#
%matplotlib inline

Load utility codes

In [ ]:
from codes.pre_processing import *
from codes.segmentation import *
from codes.utils import *
from codes.training import *
from codes.model import *

Training/Validation split

In [ ]:
data_root_dir = '../training_set'
train_file_list, val_file_list = test_val_split_v2(data_root_dir, train_percentage = 90)
ref_file = os.path.join(data_root_dir, 'REFERENCE.csv')
#
list_of_training_files = np.array(train_file_list)
list_of_validation_files = np.array(val_file_list)

Define model graph

In [ ]:
# 1000 x 12 batches
input_size = 12
time_steps = 1000
num_classes = 9
hidden_size = 100
num_hidden = 2
output_size = 9
keep_prob = 0.5  # dropout
inputs = tf.placeholder(tf.float32, [None, time_steps, input_size])
labels = tf.placeholder(tf.int32, [None])
seq_length = tf.placeholder(tf.int32, [None])
#
is_training = True # set it to true at first
#
def RNN_bidirectional(input_tensor, Training):
    with tf.variable_scope("recurrent", initializer = tf.contrib.layers.variance_scaling_initializer()):
        cell = tf.nn.rnn_cell.BasicLSTMCell
        cells_fw = [cell(hidden_size) for _ in range(num_hidden)]
        cells_bw = [cell(hidden_size) for _ in range(num_hidden)]
        cells_fw = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob = keep_prob if Training is True else 1.0) for cell in cells_fw]
        cells_bw = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob = keep_prob if Training is True else 1.0) for cell in cells_bw]
        _, states_fw, states_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                cells_fw=cells_fw,
                cells_bw=cells_bw,
                inputs=input_tensor,
                sequence_length = seq_length,
                dtype=tf.float32)
        outputs_fw = tf.concat(states_fw[-1][-1], axis = 1)
        outputs_bw = tf.concat(states_bw[-1][-1], axis = 1)
        outputs = tf.concat([outputs_fw, outputs_bw], axis = 1)
        logits = tf.squeeze(fully_connected(outputs, output_size, activation_fn = None))
        #
    return logits

In [ ]:
logits = RNN_bidirectional(inputs, Training = is_training)

Define the loss and training ops

In [ ]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = labels, logits = logits)
loss = tf.reduce_mean(cross_entropy)
#
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
gradients, variables = zip(*optimizer.compute_gradients(loss))
# gradient clipping - makes the training more stable
gradients = [
    None if gradient is None else tf.clip_by_norm(gradient, 5.0)
    for gradient in gradients]
training_op = optimizer.apply_gradients(zip(gradients, variables))
#
correct = tf.nn.in_top_k(logits, labels, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

Saver

In [ ]:
saver = tf.train.Saver(max_to_keep = 5, keep_checkpoint_every_n_hours = 1)
save_dir = './model'
#
model_name_prefix = 'model.ckpt'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

Initialize variables

In [ ]:
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

Begin training

In [ ]:
with tf.Session()  as sess:
    
    sess.run(init_op)
    path = tf.train.get_checkpoint_state(save_dir)
    if path is None:
        global_step = 0
    else:
        global_step = int(path.model_checkpoint_path.split('-')[-1])
    #
    
    if path is None:
        sess.run(init_op)
    else:
        saver.restore(sess, path.model_checkpoint_path)
    
    for step in range(20000):
        #
        data_numpy, labels_numpy, seq_length_numpy = sample_batch_for_training(list_of_training_files, ref_file)
        _, loss_value, logits_value = sess.run([training_op, loss, logits], feed_dict = {inputs: data_numpy, labels: labels_numpy, seq_length: seq_length_numpy})
        #
        if step % 20 == 0:
            print('current iteration: {}'.format(step + global_step))
            print('loss value: {}'.format(loss_value))
            acc_train = sess.run(accuracy, feed_dict = {inputs: data_numpy, labels: labels_numpy, seq_length: seq_length_numpy})
            is_training = False  # set to false before evaluating the test accuracy
            sub = np.random.randint(len(list_of_validation_files))
            data_val, labels_val, seq_length_val = sample_batch(list_of_validation_files[sub], ref_file, mode = 'training')
            acc_val = sess.run(accuracy, feed_dict = {inputs: data_val, labels: labels_val, seq_length: seq_length_val})
            is_training = True  # set to true again for the subsequent iterations
            #
            print(step, "Training accuracy:", acc_train, "Test accuracy", acc_val)
            try:
                loss_list.append(loss_value)
                acc_train_list.append(acc_train)
                acc_test_list.append(acc_val)
            except:
                loss_list = list()
                acc_train_list = list()
                acc_test_list = list()
                loss_list.append(loss_value)
                acc_train_list.append(acc_train)
                acc_test_list.append(acc_val)
        save_path = saver.save(sess, os.path.join(save_dir, model_name_prefix), global_step = step + global_step + 1) # model count begins with 1
        print("Model saved in path: %s" % save_path)
        
        if loss_value < 0.10:
            break

Plot training curves

In [ ]:
import matplotlib.pyplot as plt
plt.figure(num = 1)
plt.plot(range(0, step, 20), acc_train_list, 'k', range(0, step, 20), acc_test_list, 'b')
plt.figure(num = 2)
plt.plot(range(0, step, 20), loss_list, 'r')